In [40]:
import json
import pandas as pd
from spotify_client import *

In [6]:
f = open("challenge_set.json")
data = json.load(f)
f.close

<function TextIOWrapper.close()>

In [7]:
playlists = [pl for pl in data["playlists"] if len(pl["tracks"]) > 0]

In [32]:
playlists, track_ids, album_ids, artist_ids = [], set(), set(), set()

for item in data["playlists"][:3000]:
    if len(item["tracks"]) > 0 and "name" in item and item["name"].isalpha():
        pid = item["pid"]
        name = item["name"]
        for track in item["tracks"]:
            track_id = track["track_uri"].split(":")[-1]
            album_id = track["album_uri"].split(":")[-1]
            artist_id = track["artist_uri"].split(":")[-1]

            playlists.append([item["pid"], item["name"], track_id])
            track_ids.add(track_id)
            album_ids.add(album_id)
            artist_ids.add(artist_id)

In [68]:
playlists = pd.DataFrame(playlists, columns = ["playlist_id", "playlist_name", "track_id"])
playlists.to_csv("playlists.csv", index=False)

Get data from Spotify

In [41]:
client_id = "bfd320eaa2cf4397bf9f4a23159fe35b"
client_secret = "5a7bc7833a9341149a73849fd5a953ca"
spotify = SpotifyAPI(client_id, client_secret)

In [52]:
tracks = []
for track_id in track_ids:
    data = spotify.get_resource(query=f"tracks/{track_id}")
    track_name = data['name']
    popularity = data['popularity']
    album_id = data['album']['id']
    tracks.append([track_id, track_name, popularity, album_id])
tracks = pd.DataFrame(tracks, columns = ["track_id", "track_name", "popularity", "album_id"])

In [69]:
tracks = pd.DataFrame(tracks, columns = ["track_id", "track_name", "popularity", "album_id"])
tracks.to_csv("tracks.csv", index=False)

In [85]:
albums = []
for album_id in album_ids:
    data = spotify.get_resource(query=f"albums/{album_id}")
    album_name = data['name']
    total_tracks = data['total_tracks']
    popularity = data['popularity']
    artist_id = data['artists'][0]['id']
    release_date = data['release_date']
    albums.append([album_id, album_name, total_tracks, popularity, artist_id, release_date])

In [86]:
albums = pd.DataFrame(albums, columns = ['album_id', 'album_name', 'total_tracks', 'popularity', 'artist_id', 'release_date'])
albums.to_csv("albums.csv", index=False)

In [80]:
artists = []
for artist_id in artist_ids:
    data = spotify.get_resource(query=f"artists/{artist_id}")
    artist_name = data['name']
    popularity = data['popularity']
    genre1 = data['genres'][0] if len(data['genres']) > 0 else None
    genre2 = data['genres'][1] if len(data['genres']) > 1 else None
    artists.append([artist_id, artist_name, popularity, genre1, genre2])

In [81]:
artists = pd.DataFrame(artists, columns = ['artist_id', 'artist_name', 'popularity', 'genre1', 'genre2'])
artists.to_csv("artists.csv", index=False)